In [1]:
rm( list=ls() )  #remove all objects
gc()             #garbage collection

require("data.table")
require("lightgbm")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,604544,32.3,1292120,69.1,1292120,69.1
Vcells,1086978,8.3,8388608,64.0,1632674,12.5


Loading required package: data.table

Loading required package: lightgbm

Loading required package: R6



In [2]:
#setwd("C:/_MCD/Labo1/code")  #Establezco el Working Directory
setwd( "~/buckets/b1" )

kdataset       <- "./datasets/competencia2_2022_limpieza1.csv.gz"
ksemilla_azar  <- 100019
ktraining      <- c( 202001,202002,202003,202004,202005,202007,202008,202009,202010,202011,202012,202101,202102,202103 )   #periodos en donde entreno
kfuture        <- c( 202105 )   #periodo donde aplico el modelo final



In [3]:
kexperimento   <- "XG_PRED_DC"

#kmax_bin           <-    31
#klearning_rate     <-     0.0687538981
#knum_iterations    <-   418
#knum_leaves        <-   925
#kmin_data_in_leaf  <-  4730
#kfeature_fraction  <-     0.2759150797

kmax_bin           <-    31
klearning_rate     <-     0.0283731427609622
knum_iterations    <-   321
knum_leaves        <-   744
kmin_data_in_leaf  <-  2667
kfeature_fraction  <-     0.933723231292524

In [4]:
#cargo el dataset donde voy a entrenar
dataset  <- fread(kdataset, stringsAsFactors= TRUE)

#paso la clase a binaria que tome valores {0,1}  enteros
#set trabaja con la clase  POS = { BAJA+1, BAJA+2 } 
#esta estrategia es MUY importante
dataset[ , clase01 := ifelse( clase_ternaria %in%  c("BAJA+2","BAJA+1"), 1L, 0L) ]

#los campos que se van a utilizar
campos_buenos  <- setdiff( colnames(dataset), c("clase_ternaria","clase01","foto_mes") )

#establezco donde entreno
dataset[ , train  := 0L ]
dataset[ foto_mes %in% ktraining, train  := 1L ]

#creo las carpetas donde van los resultados
#creo la carpeta donde va el experimento
# HT  representa  Hiperparameter Tuning
dir.create( "./exp/",  showWarnings = FALSE ) 
dir.create( paste0("./exp/", kexperimento, "/" ), showWarnings = FALSE )
setwd( paste0("./exp/", kexperimento, "/" ) )   #Establezco el Working Directory DEL EXPERIMENTO


#dejo los datos en el formato que necesita LightGBM
dtrain  <- lgb.Dataset( data= data.matrix(  dataset[ train==1L, campos_buenos, with=FALSE]),
                        label= dataset[ train==1L, clase01] )

#aplico el modelo a los datos sin clase
dapply  <- dataset[ foto_mes== kfuture ]

ERROR: Error: no se puede ubicar un vector de tamaño  2.5 Gb


In [ ]:
dapply[,foto_mes:=NULL]

In [ ]:
#genero el modelo
#estos hiperparametros  salieron de una laaarga Optmizacion Bayesiana
modelo  <- lgb.train( data= dtrain,
                      param= list( objective=          "binary",
                                   max_bin=            kmax_bin,
                                   learning_rate=      klearning_rate,
                                   num_iterations=     knum_iterations,
                                   num_leaves=         knum_leaves,
                                   min_data_in_leaf=   kmin_data_in_leaf,
                                   feature_fraction=   kfeature_fraction,
                                   seed=               ksemilla_azar
                                  )
                    )

In [ ]:
#--------------------------------------
#ahora imprimo la importancia de variables
tb_importancia  <-  as.data.table( lgb.importance(modelo) ) 
archivo_importancia  <- "impo.csv"

fwrite( tb_importancia, 
        file= archivo_importancia, 
        sep= "\t" )

In [ ]:
#aplico el modelo a los datos nuevos
prediccion  <- predict( modelo, 
                        data.matrix( dapply[, campos_buenos, with=FALSE ])                                 )

#genero la tabla de entrega
tb_entrega  <-  dapply[ , list( numero_de_cliente ) ]
tb_entrega[  , prob := prediccion ]

#grabo las probabilidad del modelo
#fwrite( tb_entrega,
#        file= "prediccion.txt",
#        sep= "\t" )

#ordeno por probabilidad descendente
setorder( tb_entrega, -prob )


#genero archivos con los  "envios" mejores
#deben subirse "inteligentemente" a Kaggle para no malgastar submits
#cortes <- seq( 3000, 15000, by=500 )
#for( envios  in  cortes )
#{
#  tb_entrega[  , Predicted := 0L ]
#  tb_entrega[ 1:envios, Predicted := 1L ]

#  fwrite( tb_entrega[ , list(numero_de_cliente, Predicted)], 
#          file= paste0(  kexperimento, "_", envios, ".csv" ),
#          sep= "," )
#}

In [ ]:
prob_corte <- 0.0372744903634901
tb_entrega[  , Predicted := 0L ]
tb_entrega[ prob >= prob_corte, Predicted := 1L ]

  fwrite( tb_entrega[ , list(numero_de_cliente, Predicted)], 
          file= paste0(  kexperimento, "_prob_corte.csv" ),
          sep= "," )
